In [572]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import scipy as sc
import keras
import tensorflow as tf
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from keras import Sequential
from xgboost import XGBRegressor

In [573]:
df_mean = pd.read_csv('data/df_mean.csv', index_col=0)
df_mean.head()

,Rbin,mean,v,npt
0,67600,-245.015625,0.7,2.0
1,67100,49.808594,0.7,2.0
2,66900,186.148438,0.7,1.0
3,67400,413.269531,0.7,2.0
4,67000,278.406250,0.7,1.0


In [574]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler

import tensorflow as tf
import statsmodels.api as sm

from keras import Sequential

from xgboost import XGBRegressor

In [575]:
data_mean = df_mean.dropna()
data_mean = data_mean.astype(float)
data_mean.head()

,Rbin,mean,v,npt
0,67600.0,-245.015625,0.7,2.0
1,67100.0,49.808594,0.7,2.0
2,66900.0,186.148438,0.7,1.0
3,67400.0,413.269531,0.7,2.0
4,67000.0,278.406250,0.7,1.0


In [1150]:
# Plot R_delta_mean vs R_bin
fig = go.Figure()

# for val in (1, 1.4, 1.8):
for val in df_mean['v'].unique():
    data = df_mean.loc[df_mean['v']==val]
    fig.add_trace(go.Scatter(y = data['mean'],
                             x = data['Rbin'],
                             mode='markers',
                             name=f'{val}V'))
fig.update_layout(
    height=600,
    font_size=18,
    title="\u0394R\u03BC vs Rbin",
    xaxis_title="Rbin (\u03A9)",
    yaxis_title="\u0394R\u03BC (\u03A9)",
    # xaxis_range=[60000,250000],
    # yaxis_range=[-15000,22000],
)

In [1167]:
vval = 1.45
# Prepare data of R_delta_mean for training and testing
X_train = data_mean.loc[(data_mean['v']<vval), ['Rbin', 'v', 'npt']]
X_test = data_mean.loc[(data_mean['v']>=vval), ['Rbin', 'v', 'npt']]
Y_train = data_mean.loc[(data_mean['v']<vval), 'mean']
Y_test = data_mean.loc[(data_mean['v']>=vval), 'mean']

In [1168]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [1169]:
pf3 = Pipeline(steps=[
    ("scaler", StandardScaler()),                                                         # Scale the transformed features
    ("poly3", PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)),  # Apply polynomial transformation
])

ct = ColumnTransformer(transformers=[
    ("num", pf3, ["Rbin", "v", "npt"]),
])

In [1170]:
x_train_pf = ct.fit_transform(x_train)
x_val_pf = ct.transform(x_val)
X_test_pf = ct.transform(X_test)

In [1171]:
model = LinearRegression(fit_intercept=False)
model.fit(x_train_pf, y_train)
model.score(x_train_pf, y_train)

0.6469444983581731

In [1172]:
# model0 = LinearRegression(fit_intercept=False)
# model0.fit(x_train_pf, y_train)
# y_train_pred = model0.predict(x_train_pf)
# model0.score(x_train_pf, y_train)

# residuals = y_train - y_train_pred
# residual_variances = np.var(residuals)
# weights = 1/abs(residuals)
# weights

# model = sm.WLS(y_train, x_train_pf, weights=weights)
# model = model.fit()
# model.rsquared

In [1173]:
# model = MLPRegressor(random_state=42, max_iter=500, solver='adam', learning_rate='adaptive', alpha=0.2, early_stopping=True, n_iter_no_change=20)
# model = model.fit(x_train_pf, y_train)
# model.score(x_train_pf, y_train)


In [1174]:
# pd.Series(model.loss_curve_, index=range(0,model.n_iter_)).plot()

In [1175]:
# Validation score
y_val_pred = model.predict(x_val_pf)
r2_score(y_val, y_val_pred)

0.6397053272607474

In [1176]:
# Test score
y_test_pred = model.predict(X_test_pf)
r2_score(Y_test, y_test_pred)

0.667431803647192

In [1177]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(y = Y_test,
#                             x = X_test['Rbin'],
#                             marker=dict(color='blue'),
#                             mode='markers',
#                             name=f'{X_test['v'].mean()}V'
#                             ))
# fig.add_trace(go.Scatter(y = y_test_pred,
#                             x = X_test['Rbin'],
#                             line=dict(color='red'),
#                             mode='markers',
#                             name='Prediction'
#                             ))
# fig.update_layout(
# title="\u0394R\u03C3 vs Rbin",
# xaxis_title="Rbin",
# yaxis_title="\u0394R\u03C3"
# )

In [1178]:
plot_df = pd.DataFrame()
# test data
plot_df['v'] = X_test['v']
plot_df['Rbin'] = X_test['Rbin']
plot_df['Y_test'] = Y_test
plot_df['y_test_pred'] = y_test_pred
plot_df = plot_df.sort_values('v')
plot_df.head()

,v,Rbin,Y_test,y_test_pred
6418,1.45,100300.0,5415.054688,10432.150457
6884,1.45,173000.0,-617.631114,-750.295470
6885,1.45,114400.0,8052.429688,5660.210658
6886,1.45,121600.0,5878.902344,3865.236975
6887,1.45,129300.0,169.580729,2354.108069


In [1179]:
colours = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52',
           '#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'
           ]
fig = go.Figure()

# for val, col in zip((1.5, 1.8), colours):
for val, col in zip(plot_df['v'].unique(), colours):
    data = plot_df.loc[plot_df['v']==val]
    fig.add_trace(go.Scatter(y = data['Y_test'],
                             x = data['Rbin'],
                            #  marker=dict(color='medium turquoise'),
                             marker=dict(color=col),
                             mode='markers',
                             name=f'{val}V'
                             ))
    fig.add_trace(go.Scatter(y = data['y_test_pred'],
                             x = data['Rbin'],
                             line=dict(color='red'),
                             mode='markers',
                             name='prediction'
                             ))
fig.update_layout(
    font_size=16,
    height=600,
    title="\u0394R\u03BC vs Rbin",
    xaxis_title="Rbin",
    yaxis_title="\u0394R\u03BC"
)

In [1180]:
plot_df_val = pd.DataFrame()
# validation data
plot_df_val['v'] = x_val['v']
plot_df_val['Rbin'] = x_val['Rbin']
plot_df_val['y_val'] = y_val
plot_df_val['y_val_pred'] = y_val_pred

plot_df_val = plot_df_val.sort_values('v')

In [1181]:
colours = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52',
           '#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'
           ]
fig = go.Figure()

for val, col in zip(plot_df_val['v'].unique(), colours):
    data = plot_df_val.loc[plot_df_val['v']==val]
    fig.add_trace(go.Scatter(y = data['y_val'],
                             x = data['Rbin'],
                             marker=dict(color=col),
                             mode='markers',
                             name=f'{val}V'
                             ))
    fig.add_trace(go.Scatter(y = data['y_val_pred'],
                             x = data['Rbin'],
                             line=dict(color='red'),
                             mode='markers',
                             name=f'{val}V'
                             ))
fig.update_layout(
    title="\u0394R\u03BC vs Rbin",
    xaxis_title="Rbin",
    yaxis_title="\u0394R\u03BC"
)

In [1186]:
fig = go.Figure()

for val1, col1 in zip(plot_df_val['v'].unique(), colours):
    data1 = plot_df_val.loc[plot_df_val['v']==val1]
    # fig.add_trace(go.Scatter(y = data1['y_val'],
    #                          x = data1['Rbin'],
    #                          marker=dict(color='red'),
    #                          mode='markers',
    #                          name=f'{val1}V'
    #                          ))
    fig.add_trace(go.Scatter(y = data1['y_val_pred'],
                             x = data1['Rbin'],
                             marker=dict(color=col1),
                             mode='markers',
                             name=f'{val1}V'
                             ))
for val, col in zip(plot_df['v'].unique(), colours):
    data = plot_df.loc[plot_df['v']==val]
    # fig.add_trace(go.Scatter(y = data['Y_test'],
    #                          x = data['Rbin'],
    #                          marker=dict(color=col),
    #                          mode='markers',
    #                          name=f'{val}V'
    #                          ))
    fig.add_trace(go.Scatter(y = data['y_test_pred'],
                             x = data['Rbin'],
                             marker=dict(color='red'),
                             mode='markers',
                             name='prediction'
                             ))
fig.update_layout(
    font_size =16,
    height =600,
    title="\u0394R\u03BC vs Rbin",
    xaxis_title="Rbin",
    yaxis_title="\u0394R\u03BC",
    yaxis_range=[-15000,22000],
)